In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import sys



import sklearn.metrics as metrics

import argparse
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings(action='ignore')


import os


In [2]:
def random_forest(features,label,output_loc,bootstrap_num = 100,data_cut=0,feature_size=0):

    
    performance = np.zeros((bootstrap_num,))
    
    
    if feature_size:
        feature_cut = features.shape[1]-np.abs(feature_size)
    else:
        feature_cut = 0
    
    feature_importance = np.zeros((bootstrap_num,features.shape[1]-feature_cut))
    feature_importance_rank = np.zeros((bootstrap_num,features.shape[1]-feature_cut))
    
    
    for bootstrap in range(bootstrap_num):        
        if data_cut:
            
            features_cut,trash_a ,label_cut,trash_b  = train_test_split(features, label, train_size=(data_cut/label.shape[0]),shuffle=True,stratify=label,random_state=bootstrap)
            training_features, test_features, training_label, test_label = train_test_split(features_cut, label_cut, train_size=0.8,shuffle=True,stratify=label_cut,random_state=bootstrap)
 
        elif feature_size:
            if feature_size<0:
                training_features, test_features, training_label, test_label = train_test_split(features[:,features.shape[1]+feature_size:], label, train_size=0.8,shuffle=True,stratify=label,random_state=bootstrap)
            else:
                training_features, test_features, training_label, test_label = train_test_split(features[:,:feature_size], label, train_size=0.8,shuffle=True,stratify=label,random_state=bootstrap)

            
                
        else:   
            training_features, test_features, training_label, test_label = train_test_split(features, label, train_size=0.8,shuffle=True,stratify=label,random_state=bootstrap)
        X_train = pd.DataFrame(training_features)
        X_test = pd.DataFrame(test_features)
        y_train =  training_label
        y_test  = test_label 



        model_randomforest = RandomForestClassifier(random_state= 0)    
        model_randomforest.fit(X_train, y_train)
        predicted = model_randomforest.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds = metrics.roc_curve(y_test,predicted, pos_label=1)
        performance[bootstrap] = metrics.auc(fpr, tpr)
        
        feature_importance[bootstrap,:] = np.abs(model_randomforest.feature_importances_.reshape(-1,))
        feature_importance_rank[bootstrap,:] = features.shape[1]- feature_importance[bootstrap].argsort().argsort()
        # argsort.argsort는 각 feature의 위치에 해당 feautre의 순위에 반대되는, 즉, 1위는 feature size-1 크기를 배정해줌. 
        # features.shape[1]에서 뺌으로서, 제일 중요도가 큰 feature의 위치엔 1 이라는 숫자가 들어가게 됨. 
        # xth number in feature_importance_rank represents the rank of xth feature. The rank is the biggest to 1 and the smallest to feature_size. 
        
    if output_loc[-1]=="_":
        np.save(output_loc+'performance.npy',performance)
        np.save(output_loc+'feature_importance.npy',feature_importance)
        np.save(output_loc+'feature_importance_rank.npy',feature_importance_rank)
    else:
        np.save(output_loc+'/performance.npy',performance)
        np.save(output_loc+'/feature_importance.npy',feature_importance)
        np.save(output_loc+'/feature_importance_rank.npy',feature_importance_rank)
    return performance,feature_importance,feature_importance_rank

In [3]:
def data_cut(data_name): 
    
    
    features = np.load("../dataset/3.preprocessed/%s/features.npy"%data_name)
    label = np.load("../dataset/3.preprocessed/%s/label.npy"%data_name)
    
    performance_best = np.average(np.load("../result/%s/performance.npy"%data_name))
    
#     min_data_num = np.int(1/(np.sum(label)/label.shape[0]))+1
    
    
    
    
    if not 'data_cut' in os.listdir(os.getcwd()+"/../result/%s/"%data_name):
        os.mkdir("../result/%s/data_cut"%data_name)
        
    grid = np.array(([[label.shape[0],performance_best]]))
    np.save("../result/%s/data_cut/grid.npy"%data_name,grid)
#     else:
#         grid = np.load("../result/%s/data_cut/grid.npy"%data_name)
    
    current = 0 
    while(True):
        if grid.shape[0] ==current+1:
            if grid[current,1] <=0.55:
                print("STOP by average AUC under 0.55")
                break
#             elif grid[current,0] <min_data_num*2:
#                 break
            else:
                datasize = int(np.round(0.5*grid[current,0]))
                # what if AUC<0.55 by 0.5? 

                try:
                    performance,trash_a,trash_b = random_forest(features,label,"../result/%s/data_cut/%s_"%(data_name,datasize),100,datasize)  
                except:
                    print("AUC not available ")
                    break
                
                grid = np.append(grid,[[datasize,np.average(performance)]],axis=0)
                np.save("../result/%s/data_cut/grid.npy"%data_name,grid)
            
            
            
        if grid[current,1]-grid[current+1,1]<=0.05:
            current +=1
        else:
            
            if grid[current,0]-grid[current+1,0] ==1:
                current+=1
                continue
            
            datasize = int((grid[current,0]+grid[current+1,0])/2)
            performance,trash_a,trash_b = random_forest(features,label,"../result/%s/data_cut/%s_"%(data_name,datasize),100,datasize)    
            grid = np.append(np.append(grid[0:current+1,:].reshape(-1,2),
                                       np.array([[datasize,np.average(performance)]]),axis=0),
                             grid[current+1:,:].reshape(-1,2),axis=0)
            np.save("../result/%s/data_cut/grid.npy"%data_name,grid)
            
            
    return grid


In [ ]:
#                 no_more_division = False
#                 while(True):
#                     try:
#                         performance,trash_a,trash_b = random_forest(features,label,"../result/%s/data_cut/%s_"%(data_name,datasize),100,datasize)  
#                         break
#                     except:
#                         print("AUC not available ")
#                         if datasize==int(np.round((datasize+int(grid[current,0]))/2)):
#                             no_more_division=True
#                             break
#                         datasize = int(np.round((datasize+int(grid[current,0]))/2))
                
#                 if no_more_division:
#                     break
#                 grid = np.append(grid,[[datasize,np.average(performance)]],axis=0)

In [5]:
def feature_cut(data_name): 
    
    features = np.load("../dataset/3.preprocessed/%s/features.npy"%data_name)
    label = np.load("../dataset/3.preprocessed/%s/label.npy"%data_name)
    
    performance_best = np.average(np.load("../result/%s/performance.npy"%data_name))
    
    
    if not 'feature_cut' in os.listdir(os.getcwd()+"/../result/%s/"%data_name):
        os.mkdir("../result/%s/feature_cut"%data_name)
        
    grid = np.array(([[features.shape[1],performance_best]]))
    np.save("../result/%s/feature_cut/grid.npy"%data_name,grid)

    
    current = 0 
    while(True):
        # 
        if grid.shape[0] ==current+1:
            if grid[current,1] <=0.55:
                break
#             elif grid[current,0] <=features.shape[1]:
#                 break
            elif grid[current,0] <= 2:
                break

            else:
                featuresize = int(0.5*grid[current,0])                
                
                try:
                    performance,trash_a,trash_b = random_forest(features,label,"../result/%s/feature_cut/%s_"%(data_name,featuresize),100,0,-featuresize)   
                except:
                    print("AUC not available ")
                    break
                
#                 performance,trash_a,trash_b = random_forest(features,label,"../result/%s/feature_cut/%s_"%(data_name,featuresize),100,0,-featuresize)    
                grid = np.append(grid,[[featuresize,np.average(performance)]],axis=0)
                np.save("../result/%s/feature_cut/grid.npy"%data_name,grid)
            
            
        if grid[current,1]-grid[current+1,1]<=0.05:
            current +=1
        else:
            
            if abs(grid[current,0]-grid[current+1,0]) ==1:
                current+=1
                continue
            
            featuresize = int((grid[current,0]+grid[current+1,0])/2)
            performance,trash_a,trash_b = random_forest(features,label,"../result/%s/feature_cut/%s_"%(data_name,featuresize),100,0,-featuresize)    
            grid = np.append(np.append(grid[0:current+1,:].reshape(-1,2),
                                       np.array([[featuresize,np.average(performance)]]),axis=0),
                             grid[current+1:,:].reshape(-1,2),axis=0)
            np.save("../result/%s/feature_cut/grid.npy"%data_name,grid)
            
    return grid


In [7]:

# # loc = "real_1_IBD"
# # loc = "real_1_IBD_lowcorr_0.5"
# # loc = "real_1_IBD_lowcorr_0.3"
# # loc = "real_1_IBD_lowcorr_0.2"
# loc = "real_2_RSV"
# # loc = "real_3_perio_health"
# # loc = "real_4_DM"
# # loc = "real_5_DR"
# # loc = "real_6_HF"
# # loc = "real_7_PK"
# # loc = "real_8_TS"
# # loc = "real_9_HT"
# # loc = "real_10_perio_nutrition"
# # loc = "real_11_BC"


# feature_cut("%s"%loc)
# data_cut("%s"%loc)

array([[2.65400000e+03, 9.68915528e-01],
       [1.32700000e+03, 9.63723427e-01],
       [6.63000000e+02, 9.60212585e-01],
       [3.31000000e+02, 9.54650974e-01],
       [1.65000000e+02, 9.36464286e-01],
       [8.20000000e+01, 9.09285714e-01],
       [4.10000000e+01, 8.66250000e-01],
       [2.00000000e+01, 8.40000000e-01],
       [1.70000000e+01, 8.08333333e-01],
       [1.60000000e+01, 8.48333333e-01],
       [1.50000000e+01,            nan],
       [1.40000000e+01,            nan],
       [1.30000000e+01,            nan],
       [1.20000000e+01,            nan],
       [1.10000000e+01, 8.02500000e-01],
       [1.00000000e+01,            nan]])

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.